In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [3]:
# Run this cell only if you are using Colab with Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Run this cell only if you are using Colab with Drive
!unzip '/content/drive/My Drive/UCMerced_LandUse.zip'

Archive:  /content/drive/My Drive/UCMerced_LandUse.zip
   creating: UCMerced_LandUse/
   creating: UCMerced_LandUse/test/
   creating: UCMerced_LandUse/test/agricultural/
  inflating: UCMerced_LandUse/test/agricultural/agricultural01.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural08.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural20.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural26.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural30.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural33.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural37.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural40.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural41.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural72.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural79.tif  
  inflating: UCMerced_LandUse/test/agricultural/agricultural82.tif  
 

# Example: Multi-class Land Use classification
## Uc Merced Land Use Dataset

### Directory structure

    - UCMerced_LandUse/
        - training/
            - agricultural/
                - img1, img2, …, imgN
            - …
            - parkinglot/ 
                - img1, img2, ... , imgN
        - validation/
            - agricultural/
                - img1, img2, …, imgN
            - …
            - parkinglot/ 
                - img1, img2, ... , imgN
        - test/
            - agricultural/
                - img1, img2, …, imgN
            - …
            - parkinglot/ 
                - img1, img2, ... , imgN

In [6]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input 

apply_data_augmentation = False

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        preprocessing_function=preprocess_input)  # to apply vgg normalization
else:
    train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [7]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'UCMerced_LandUse')

# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256

num_classes=21

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

# Validation
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

# Test
test_dir = os.path.join(dataset_dir, 'test')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=bs, 
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

Found 1470 images belonging to 21 classes.
Found 315 images belonging to 21 classes.
Found 315 images belonging to 21 classes.


In [8]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
# Without calling the repeat function the dataset will be empty after consuming all the images
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
test_dataset = valid_dataset.repeat()

# Transfer Learning
## e.g., VGG16

In [12]:
# Load VGG16 Model

vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [ ]:
vgg.summary()

In [19]:
# Create Model
# ------------

finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
    
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 512)               0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 21)                10773     
Total params: 14,988,117
Trainable params: 7,352,853
Non-trainable params: 7,635,264
___________________________________

## Prepare the model for training

In [20]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training with callbacks

In [21]:
import os
from datetime import datetime

cwd = '/content/drive/My Drive/Keras3'

exps_dir = os.path.join(cwd, 'transfer_learning_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

# reduce learning rate
# --------------
reduce_learning_rate = True
if reduce_learning_rate:
    reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                         patience=2, verbose=1)
    callbacks.append(reduce_learning_rate)

model.fit(x=train_dataset,
          epochs=30,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)


Epoch 1/30
46/46 [==============================] - 17s 372ms/step - loss: 2.5331 - accuracy: 0.2823 - val_loss: 1.1634 - val_accuracy: 0.7016
Epoch 2/30
46/46 [==============================] - 14s 308ms/step - loss: 0.9846 - accuracy: 0.6993 - val_loss: 0.4176 - val_accuracy: 0.8571
Epoch 3/30
46/46 [==============================] - 14s 303ms/step - loss: 0.5071 - accuracy: 0.8435 - val_loss: 0.2720 - val_accuracy: 0.9111
Epoch 4/30
46/46 [==============================] - 14s 306ms/step - loss: 0.2736 - accuracy: 0.9204 - val_loss: 0.2244 - val_accuracy: 0.9333
Epoch 5/30
46/46 [==============================] - 14s 298ms/step - loss: 0.1834 - accuracy: 0.9503 - val_loss: 0.2255 - val_accuracy: 0.9206
Epoch 6/30
46/46 [==============================] - 14s 299ms/step - loss: 0.1134 - accuracy: 0.9639 - val_loss: 0.1451 - val_accuracy: 0.9429
Epoch 7/30
46/46 [==============================] - 14s 309ms/step - loss: 0.1005 - accuracy: 0.9680 - val_loss: 0.1092 - val_accuracy: 0.9587

In [ ]:
# run this if you want to save the whole model
#model.save('/content/drive/My Drive/path')

## Test model

In [23]:
# use this if you want to restore saved model
# model.load_weights('/path/to/checkpoint')  

eval_out = model.evaluate(x=test_dataset,
                          steps=len(test_gen),
                          verbose=0)
eval_out

[0.06702920794487, 0.9809523820877075]